In [1]:
import os
import pickle
import email_read_util

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/workstationhp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
DATA_DIR = 'datasets/trec07p/data/'
LABELS_FILE = 'datasets/trec07p/full/index'
TRAINING_SET_RATIO = 0.7

In [3]:
labels = {}
spam_words = set()
ham_words = set()

In [5]:
# Read the labels
with open(LABELS_FILE) as f:
    for line in f:
        line = line.strip()
        label, key = line.split()
        labels[key.split('/')[-1]] = 1 if label.lower() == 'ham' else 0

In [6]:
# Split corpus into train and test sets
filelist = os.listdir(DATA_DIR)
X_train = filelist[:int(len(filelist)*TRAINING_SET_RATIO)]
X_test = filelist[int(len(filelist)*TRAINING_SET_RATIO):]

In [10]:
if not os.path.exists('blacklist.pkl'):
    for filename in X_train:
        path = os.path.join(DATA_DIR, filename)
        if filename in labels:
            label = labels[filename]
            stems = email_read_util.load(path)
            if not stems:
                continue
            if label == 1:
                ham_words.update(stems)
            elif label == 0:
                spam_words.update(stems)
            else:
                continue
    blacklist = spam_words - ham_words
    pickle.dump(blacklist, open('blacklist.pkl', 'wb'))
else:
    blacklist = pickle.load(open('blacklist.pkl', 'rb') )

print('Blacklist of {} tokens successfully built/loaded'.format(len(blacklist)))

Blacklist of 98136 tokens successfully built/loaded


In [13]:
from nltk.corpus import words
word_set = set(words.words())
word_set.intersection(blacklist)

{'tamer',
 'degum',
 'astrakhan',
 'coachman',
 'mank',
 'dha',
 'cine',
 'thrasher',
 'catchup',
 'eer',
 'glacier',
 'laudanum',
 'magnolia',
 'passeriform',
 'anteroom',
 'tubman',
 'corpora',
 'octillion',
 'precinct',
 'doorway',
 'hame',
 'caiman',
 'abas',
 'depositor',
 'skil',
 'talak',
 'aquavit',
 'discontented',
 'pina',
 'bairn',
 'pergola',
 'bisect',
 'soundless',
 'cuttyhunk',
 'waster',
 'wend',
 'sleet',
 'remodel',
 'smidgen',
 'gerent',
 'bulkhead',
 'clockwork',
 'lightroom',
 'bungalow',
 'fana',
 'individualist',
 'amarillo',
 'retook',
 'mone',
 'linnet',
 'affray',
 'spook',
 'overhang',
 'rook',
 'chino',
 'cyclopean',
 'terminal',
 'holdup',
 'mellow',
 'howler',
 'slat',
 'freehold',
 'indebt',
 'fandom',
 'turban',
 'hinterland',
 'rococo',
 'ebb',
 'higdon',
 'sudd',
 'ruffian',
 'patten',
 'earplug',
 'seraphina',
 'stela',
 'cotta',
 'unlearn',
 'larviform',
 'ordinar',
 'raptor',
 'galt',
 'systematist',
 'quagga',
 'cutthroat',
 'abbey',
 'trist',
 're

In [14]:
fp = 0
tp = 0
fn = 0
tn = 0

for filename in X_test:
    path = os.path.join(DATA_DIR, filename)
    if filename in labels:
        label = labels[filename]
        stems = email_read_util.load(path)
        if not stems:
            continue
        stems_set = set(stems)
        if stems_set & blacklist:
            if label == 1:
                fp = fp + 1
            else:
                tp = tp + 1
        else:
            if label == 1:
                tn = tn + 1
            else:
                fn = fn + 1


In [15]:


from IPython.display import HTML, display
conf_matrix = [[tn, fp],
               [fn, tp]]
display(HTML('<table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join('<td>{}</td>'.format(
        '</td><td>'.join(str(_) for _ in row)) 
                     for row in conf_matrix))))



6956,743
5718,7419


In [16]:


count = tn + tp + fn + fp
percent_matrix = [["{:.1%}".format(tn/count), "{:.1%}".format(fp/count)],
                  ["{:.1%}".format(fn/count), "{:.1%}".format(tp/count)]]
display(HTML('<table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join('<td>{}</td>'.format(
        '</td><td>'.join(str(_) for _ in row)) 
                     for row in percent_matrix))))



33.4%,3.6%
27.4%,35.6%


In [17]:
print("Classification accuracy: {}".format("{:.1%}".format((tp+tn)/count)))

Classification accuracy: 69.0%
